# Тексификация коротких текстов с математическими выражениями

Тексификация — это преобразование исходного текста, содержащего математические сущности, выраженные в смеси полусловесных формулировок и/или различных математических языков разметки, в текст, приведенный в соответствие с правилами системы компьютерной вёрстки LaTeX для русского языка, обладающий большим уровнем восприятия, в котором выделены математические сущности.

## Установка

In [1]:
!git clone https://github.com/basic-go-ahead/emma -b st1
!pip install -q -r ./emma/requirements.txt

Cloning into 'emma'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 37 (delta 15), reused 27 (delta 8), pack-reused 0
Receiving objects: 100% (37/37), 16.24 KiB | 8.12 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
import torch
import sys

In [3]:
sys.path.append('./emma')

In [4]:
from emma import Texificator


## Создание тексификатора

Преобразование входного текста, содержащего математические выражения, выполяет объект типа `Texificator`:

In [5]:
texificator = Texificator()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/511k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.13M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/3.08k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/937M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

По умолчанию тексификатор инкапсулирует модель `basic-go/rut5-base-texificator-st1` и упрощает взаимодействие с ней.

При инициализации тексификатора с помощью параметра `device` можно задать устройство, на котором будут выполняться вычисления. По умолчанию параметр `device` выбирает графический процессор, если он доступен, и CPU в противном случае.

## Преобразование входного текста

Для преобразования входного текста, содержащего математические выражения, необходимо использовать метод `doit` тексификатора:

In [6]:
texificator.doit("Разложить на множители выражение икс квадрат минус пять икс плюс четыре.")

'Разложить на множители выражение \\(x^{2}-5x+4\\).'

Метод `doit` принимает в качестве первого входного параметра строку, как в предыдущем примере, или список строк, которые необходимо обработать:

In [7]:
for texified in texificator.doit([
    "1/α + 1/β = 1/γ",
    "для любого ε больше нуля существует дельта больше нуля",
    "сумма от k=1 до 10 k в квадрате"
]):
  print(texified)

\(\frac{1}{\alpha} + \frac{1}{\beta} = \frac{1}{\gamma}\)
\(\forall \varepsilon>0\exists\delta>0\)
\(\sum_{k=1}^{10}k^{2}\)


Второй параметр метода `doit` тексификатора определяет тип текификации, который необходимо выполнить. Метод поддерживает следующие два типа:
- полная тексификация (`"full"`) и
- веб-тексификация (`"web"`).

По умолчанию выполняется полная тексификация, предназначенная для генерации вёрстки LaTeX для печатных документов:

In [8]:
texificator.doit("На корабле «Восток» 12 апреля 1961 года Ю.А. Гагарин совершил полет в космос. ")

'На корабле <<Восток>> 12~апреля 1961~года Ю.\\,А. Гагарин совершил полет в космос.'

Полная тексификация осуществляет преобразование кавычек в специальные команды LaTeX, а также расставляет неразрывные пробелы. Кроме того, при полной тексификации осуществляется преобразование популярных математических функций и букв греческого алфавита в их представления, характерные для русской типографии печатных документов:

In [9]:
texificator.doit(r"$ \sinh{\phi} = \frac{e^{\phi} - e^{-x}}{\phi} $")

'\\( \\sh{\\varphi} = \\frac{e^{\\varphi} - e^{-x}}{\\varphi} \\)'

Веб-тексификация предназначена для последующего рендеринга тексифицированного контента в веб. При веб-тексификации математические функции, характерные для русской типографии, обрамляются специальными командами LaTeX:

In [10]:
texificator.doit(r"$ \sinh{\phi} = \frac{e^{\phi} - e^{-x}}{\phi} $", texification_type="web")

'\\( \\operatorname{sh}{\\varphi} = \\frac{e^{\\varphi} - e^{-x}}{\\varphi} \\)'

Также тексификатор обладает методом `set_generation_params`, позволяющим передать собственные параметры генерации нейронной сети, заданные словарём.

## Поддерживаемые типы входных текстов

Тексификатор поддерживает следующие типы входных текстов:

- словесные формулировки;
- разметка на языке LaTeX, имеющая незначительные повреждения;
- разметка на языке AsciiMath;
- специальные математические символы Unicode, а также внутритекстовые дроби.

### Словесные формулировки

In [11]:
samples = [
    "чосинус икс минус шинус три игрек",
    "неопределённый интеграл жи штрих от икс дэ икс равно жи от икс плюс цэ большое",
    "криволинейный интеграл по контуру гамма большое эф(тау) дэ тау",
    "предел при эс стремящемся к минус бесконечности синус эс делить на эс плюс один"
]

for e in texificator.doit(samples):
  print(e)

\(\ch{x } - \sh{3 y }\)
\(\int g'(x)dx=g(x)+C\)
\(\oint_\gamma f(\tau)d\tau\)
\(\lim_{s\rightarrow-\infty}\frac{\sin{s}}{s}+1\)


### Повреждённая разметка

In [12]:
samples = [
    r"\frac{1{3 - x^{2 + \phi(x(t)",
    r"limt_{x \to \intfy} e^-x",
    r"$$ \frac{e^R}{R} \to 0 \qud R \to -\infty $$",
]

for e in texificator.doit(samples):
  print(e)

\(\frac{1}{3} - x^{2} + \varphi(x(t))\)
\(\lim \limits_{x \to \infty} e^{-x}\)
\[ \frac{e^R}{R} \to 0 \quad R \to -\infty \]


### Разметка на языке AsciiMath

In [13]:
samples = [
    r"sum_(i=1)^n i^3=((n(n+1))/2)^2",
    r"lim_(N->oo) sum_(i=0)^N 1/i^2",
    r"f : RR^(2) -> RR^(3)"
]

for e in texificator.doit(samples):
  print(e)

\(\sum_{i=1}^n i^3=((n(n+1))/2)^2\)
\(\lim_{N\to \infty} \sum_{i=0}^N \frac{1}{i^2}\)
\(f : \mathbb{R}^2 \to \mathbb{R}^3\)


### Специальные математические символы Unicode и внутритекстовые дроби.

In [14]:
samples = [
    "∫ ρ(x) dx",
    "∑_k |φ_k| ≤ 1",
    "α^2 + β2 ≠ γ^2",
    "π ≈ 3,14",
    "x→∞"
]

for e in texificator.doit(samples):
  print(e)

\(\int \rho(x) dx\)
\(\sum_k |\varphi_k| \le 1\)
\(\alpha^2 + \beta^2 \neq \gamma^2\)
\(\pi \approx 3,14\)
\(x\to \infty\)
